#### Bonus: Because we want to predict the trending products, so we use the  `Logistic Regression` model to classificate the product is trending or not.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import warnings 
warnings.filterwarnings('ignore')

%matplotlib inline

### <b>Read the data</b>

In [2]:
df = pd.read_csv('../../data/processed/canonical_data.csv')
df.head() 

,p_categories,p_brand,p_day_created,p_sold_quantity,p_original_price,p_discount_rate
0,7,155,659.0,0.041052,528000,49
1,84,198,974.0,7.554966,799000,46
2,84,192,1372.0,0.187097,209000,0
3,7,155,593.0,6.017157,473000,50
4,15,145,529.0,1.053469,106000,16


### <b>Create columns trending</b>
1. The products have mean 

In [3]:
col_trending = df['p_sold_quantity'] / df['p_day_created']
df['trending'] = col_trending.apply(lambda x: 1 if x > col_trending.mean() else 0)


### <b>Train test split</b>

In [4]:
X = df.drop('trending', axis='columns')
y = testLabels = df.trending

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15)

#### **View data after train test split**

In [5]:
y_train.value_counts()

1    1016
0     404
Name: trending, dtype: int64

In [6]:
y.value_counts()

1    1258
0     518
Name: trending, dtype: int64

In [7]:
104/89

1.1685393258426966

In [8]:
y_test.value_counts()

1    242
0    114
Name: trending, dtype: int64

In [9]:
X_train.shape, X_test.shape

((1420, 6), (356, 6))

In [10]:
X_train[:10]

,p_categories,p_brand,p_day_created,p_sold_quantity,p_original_price,p_discount_rate
1126,127,103,143.0,-0.343864,75800,0
1158,62,283,90.0,-0.376044,90000,3
1657,84,173,708.0,-0.189155,95000,0
184,84,182,758.0,0.533027,1000000,55
1649,97,135,747.0,-0.015881,500000,47
642,84,211,1140.0,0.513843,499000,46
905,127,179,99.0,-0.389658,139000,0
1088,28,42,762.0,-0.275174,219000,0
350,6,169,882.0,0.113456,99000,11
1235,84,192,1216.0,-0.203389,28500,0


### <b>Use the model `LogisticRegression` to predict the target variable</b>

In [11]:
def confusion_matrix_plot(y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(10, 7))
    sns.heatmap(cm, annot=True, fmt='d')
    plt.xlabel('Predicted')
    plt.ylabel('Truth')
    plt.show()

In [12]:
def log_reg(X_train, X_test, y_train, y_test, weights=-1):
    if weights == -1:
        model = LogisticRegression()
    else:
        model = LogisticRegression(class_weight={0: weights[0], 1: weights[1]})
    model.fit(X_train, y_train)
    accuracy = model.score(X_test, y_test)
    y_pred = model.predict(X_test)
    print('Accuracy: ', accuracy)
    #  confusion_matrix_plot(y_test, y_pred)
    print(classification_report(y_test, y_pred))
    return model

In [13]:
sgd = make_pipeline(StandardScaler(), LogisticRegression(penalty='l2'))

fit_intercept = [True, False]
penalty = ['l2', 'l1', 'elasticnet']
param = dict(logisticregression__penalty=penalty, logisticregression__fit_intercept = fit_intercept)

rnd_search = RandomizedSearchCV(sgd, param_distributions=param, n_iter=10, cv=5, scoring='accuracy', random_state=15)
rnd_search.fit(X_train, y_train)
rnd_search.best_params_

{'logisticregression__penalty': 'l2',
 'logisticregression__fit_intercept': True}

In [14]:
from sklearn.metrics import mean_squared_error
clf = LogisticRegression(penalty='l2', fit_intercept=True)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=15)
lr_scores = cross_val_score(clf, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)

In [15]:
print('LogisticRegression: %.3f (%.3f)' % (np.mean(lr_scores), np.std(lr_scores)))
print('Accuracy: ', accuracy_score(y_test, y_pred))

LogisticRegression: 0.831 (0.097)
Accuracy:  0.6825842696629213


- Comments:
    - Because we want to predict the trending products, so we need to create a new column `is_trending` to indicate the products are trending or not.
    - But we create is_trending column, the result is not good(overfitting).